In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199770 sha256=4c28d4f331e9021bfbae96c0f803e1685dfc82e4fbb63b64b2c331c517790e08
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import fasttext

# Replace 'path_to_model.bin' with the actual path to your .bin file
fasttext_model = fasttext.load_model('/content/drive/MyDrive/hate rate/task 1/embedding/Sinhala/cc.si.300.bin')


In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=5484645d08d88f32fa4aaf200192263bb7fba010df9f36f2d471162b872a6167
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import random
import pickle
import json
import pandas as pd
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
import os
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/hate rate/task 1/pre_text/TRAIN_SINHALA_TASK_1.csv')
test_df=pd.read_csv('/content/drive/MyDrive/hate rate/task 1/pre_text/TEST_SINHALA_TASK1.csv')

In [ ]:
unique_word=set()
for sent in df['pre_text']:
  for word in sent.split():
    unique_word.add(word)



In [ ]:
def load_word_vectors(file_path):
    word_vectors = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.rstrip().split(' ')

            word = values[0]
            if(word in unique_word):
              vector = [float(val) for val in values[1:]]
              word_vectors[word] = vector
    return word_vectors

# Replace 'path_to_vectors.vec' with the actual path to your .vec file
word_vectors = load_word_vectors('/content/drive/MyDrive/hate rate/task 1/embedding/Sinhala/cc.si.300.vec')


In [ ]:
df

,Unnamed: 0,post_id,text,label,pre_text,intlabel
0,0,726758237668659201,@USER @USER පට්ට පට පට...,NOT,පට්ට පට පට,0
1,1,915618589855617026,පරණ කෑල්ල අද වෙනකම් හිටියනම් අදට අවුරුදු 4යි. ...,HOF,පරණ කෑල්ල අද වෙනකම් හිටියනම් අදට අවුරුදු යි යා...,1
2,2,925001070430040065,යාළුවා කියලා හිතන් සර් ගේ ඔලුවට රෙද්ද දාලා නෙල...,NOT,යාළුවා කියලා හිතන් සර් ගේ ඔලුවට රෙද්ද දාලා නෙල...,0
3,3,1397219745707986955,හොඳ මිතුරියක් කතා කලා. විස්තර කතාකරමින් ඉදලා ම...,HOF,හොඳ මිතුරියක් කතා කලා විස්තර කතාකරමින් ඉදලා මේ...,1
4,4,950376113150222336,"ඔය බනින්නෙ.. හරකා, මී හරකා කිය කිය...",HOF,ඔය බනින්නෙ හරකා මී හරකා කිය කිය,1
...,...,...,...,...,...,...
7495,7495,930270216612872193,අද උදේ දැක්කා පට්ට ලස්සන හීනයක්,NOT,අද උදේ දැක්කා පට්ට ලස්සන හීනයක්,0
7496,7496,1159471424613969921,@USER කසල වෙන් කරලා දෙන්න පුරුදු වුනොත් වැඩේ ...,NOT,කසල වෙන් කරලා දෙන්න පුරුදු වුනොත් වැඩේ ලේසි වෙයි,0
7497,7497,1073271775583100928,ඒත් පබාගේ පස්ස නම්... ඉස්සෝ කොටුවක් උනත් දාන්න...,HOF,ඒත් පබාගේ පස්ස නම් ඉස්සෝ කොටුවක් උනත් දාන්න පු...,1
7498,7498,1131018244733657088,සුදුවෑන්වලට මාරම මාර බයක් තියෙන ඈයෝ අන්තවාදයට...,HOF,සුදුවෑන්වලට මාරම මාර බයක් තියෙන ඈයෝ අන්තවාදයට ...,1


In [ ]:

#finding longest sentence
maxx=0

for s in df['text']:
  s_len=len(s.split())
  if(s_len>maxx):
    sent=s
  maxx=max(s_len,maxx)

print(maxx)


118


In [ ]:
train_df,val_df=train_test_split(df,test_size=0.1,random_state=50)

In [ ]:
EMBEDDING_DIM=300

In [ ]:
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Step 1: Fit the tokenizer on the sentence corpus
tokenizer = Tokenizer(filters='',oov_token='[OOV]')

In [ ]:


tokenizer.fit_on_texts(df['pre_text'].tolist())
train_sequence=tokenizer.texts_to_sequences(train_df['pre_text'].values)
val_sequence=tokenizer.texts_to_sequences(val_df['pre_text'].values)


num_tokens=len(tokenizer.word_index)

num_tokens=len(tokenizer.word_index)+1

GLOVE_EMBEDDING_DIM=100





In [ ]:
print(num_tokens)

32535


In [ ]:
max_sequence_length = 128

In [ ]:
import fasttext
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences




# Pad sequences to a uniform length
train_padded_sequences = pad_sequences(train_sequence, maxlen=max_sequence_length, padding='post', truncating='post')
val_padded_sequences = pad_sequences(val_sequence, maxlen=max_sequence_length, padding='post', truncating='post')

# Create the embedding matrix
embedding_dim = 300  # Adjust based on your FastText embeddings
num_words = len(tokenizer.word_index) + 1  # Add 1 for the padding token
count_yes=0
embedding_matrix = np.zeros((num_words, embedding_dim))
word_embedding_not_found=[]
for word, i in tokenizer.word_index.items():
    if word in fasttext_model:
        count_yes+=1
        embedding_matrix[i] = fasttext_model[word]
    else:
      word_embedding_not_found.append(word)

# Now you can use 'embedding_matrix' as the 'weights' parameter when creating the Embedding layer in your model
print('number of words in fasttext of our tokenizer are : ',count_yes,' and total is ',num_words)

number of words in fasttext of our tokenizer are :  25390  and total is  32535


In [ ]:
train_input=train_padded_sequences
train_output=np.array(train_df['intlabel'])

val_input=val_padded_sequences
val_output=np.array(val_df['intlabel'])


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Add, Input,Concatenate,Dropout
from keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.models import Model
# Assuming you have loaded FastText embeddings into 'embedding_matrix'

# Input layer
input_layer = Input(shape=(max_sequence_length,))

# Embedding layer
embedding_layer = Embedding(input_dim=num_words,
                            output_dim=embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False)(input_layer)

# Add a 1D convolutional layer
conv_layer = Conv1D(filters=300, kernel_size=3, activation='relu', padding='same' )(embedding_layer)


#max_pooling_layer = MaxPooling1D(pool_size=2)(conv_layer)
# Second 1D convolutional layer
conv_layer_2 = Conv1D(filters=300, kernel_size=2, activation='relu', padding='same')(conv_layer)
# Apply global max pooling
residual_connection = Add()([conv_layer_2, conv_layer])
conv_layer_3 = Conv1D(filters=500, kernel_size=2, activation='relu')(residual_connection)
conv_layer_3=Dropout(0.3)(conv_layer_3)
conv_layer_4 = Conv1D(filters=300, kernel_size=1, activation='relu')(conv_layer_3)

pooling_layer = GlobalMaxPooling1D()(conv_layer_4)

# Add a dense layer
dense_layer = Dense(50, activation='relu')(pooling_layer)

# Output layer
output_layer = Dense(1, activation='sigmoid')(dense_layer)


# Create the model



In [ ]:
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)



# Print the model summary

model.summary()

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 128)]        0           []                               
                                                                                                  
 embedding_15 (Embedding)       (None, 128, 300)     529800      ['input_19[0][0]']               
                                                                                                  
 conv1d_36 (Conv1D)             (None, 128, 300)     270300      ['embedding_15[0][0]']           
                                                                                                  
 conv1d_37 (Conv1D)             (None, 128, 300)     180300      ['conv1d_36[0][0]']              
                                                                                           

In [ ]:
model.reset_states()


In [ ]:
from tensorflow.keras.optimizers import AdamW
BATCH_SIZE=32
EPOCHS=1000


learning_rate = 0.00005  # You can adjust this value
optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate)
# Compile the model
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


model.fit(train_input,
          train_output,
          batch_size=BATCH_SIZE,
          validation_data=(val_input,val_output),
          epochs=EPOCHS)

Epoch 1/1000
6/6 [==============================] - 5s 522ms/step - loss: 0.6949 - accuracy: 0.5056 - val_loss: 0.6904 - val_accuracy: 0.6000
Epoch 2/1000
6/6 [==============================] - 5s 833ms/step - loss: 0.6916 - accuracy: 0.5611 - val_loss: 0.6900 - val_accuracy: 0.5500
Epoch 3/1000
6/6 [==============================] - 3s 458ms/step - loss: 0.6888 - accuracy: 0.6167 - val_loss: 0.6898 - val_accuracy: 0.4500
Epoch 4/1000
6/6 [==============================] - 3s 468ms/step - loss: 0.6878 - accuracy: 0.6167 - val_loss: 0.6895 - val_accuracy: 0.5000
Epoch 5/1000
6/6 [==============================] - 3s 472ms/step - loss: 0.6856 - accuracy: 0.6111 - val_loss: 0.6894 - val_accuracy: 0.5500
Epoch 6/1000
6/6 [==============================] - 4s 660ms/step - loss: 0.6834 - accuracy: 0.6111 - val_loss: 0.6885 - val_accuracy: 0.5000
Epoch 7/1000
6/6 [==============================] - 4s 534ms/step - loss: 0.6829 - accuracy: 0.6222 - val_loss: 0.6878 - val_accuracy: 0.5000
Epoch 

KeyboardInterrupt: ignored

min so far val_loss=0.6400
import tensorflow as tf
from tensorflow.keras. import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Add, Input,Concatenate
from keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.models import Model
# Assuming you have loaded FastText embeddings into 'embedding_matrix'

# Input layer
input_layer = Input(shape=(max_sequence_length,))

# Embedding layer
embedding_layer = Embedding(input_dim=num_words,
                            output_dim=embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False)(input_layer)

# Add a 1D convolutional layer
conv_layer = Conv1D(filters=300, kernel_size=3, activation='relu')(embedding_layer)


max_pooling_layer = MaxPooling1D(pool_size=2)(conv_layer)
# Second 1D convolutional layer
conv_layer_2 = Conv1D(filters=300, kernel_size=3, activation='relu')(max_pooling_layer)
# Apply global max pooling
pooling_layer = GlobalMaxPooling1D()(conv_layer_2)

# Add a dense layer
dense_layer = Dense(50, activation='relu')(pooling_layer)

# Output layer
output_layer = Dense(1, activation='sigmoid')(dense_layer)


# Create the model



val loss 6300